In [2]:
# Pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

# Polars (Arrow)
from pyarrow.dataset import dataset
import polars as pl
pl.Config.set_tbl_rows(n=-1)
pl.Config.set_tbl_cols(n=-1)

# Hit API
import requests

# Tools
from itertools import chain
from datetime import datetime, timedelta
from math import pi

# Save
import pickle
import os
import pathlib

In [3]:
model_path = "API_RAW_PBP_Data.parquet"
PBP_RAW = pl.read_parquet(model_path).filter(pl.col('season_type').is_in(['R', 'P']))

In [4]:
# Path
roster_file = 'NHL_Rosters_2014_2024.csv'

# All Players - Connect To event_player_1_id, event_player_2_id, event_player_3_id, event_player_4_id, event_goalie_id, home_goalie, away_goalie
ROSTER_DF_RAW = pl.read_csv(roster_file)

ROSTER_DF = (
    ROSTER_DF_RAW
    .with_columns([
        pl.col("player_id").cast(pl.Int32),
        (pl.col("first_name").str.to_uppercase() + '.' + pl.col("last_name").str.to_uppercase()).alias('player_name'),
        pl.when((pl.col('pos_G') == 1) & (pl.col('hand_R') == 1)).then(pl.lit(1)).otherwise(pl.lit(0)).alias('G_hand_R'),
        pl.when((pl.col('pos_G') == 1) & (pl.col('hand_L') == 1)).then(pl.lit(1)).otherwise(pl.lit(0)).alias('G_hand_L')
        ])
    .select(['player_id', 'player_name', 'hand_R', 'hand_L', 'pos_F', 'pos_D', 'pos_G', 'G_hand_R', 'G_hand_L'])
    .unique()
)

ROSTER_DF_RAW.head()

player_id,first_name,last_name,pos_F,pos_D,pos_G,hand_R,hand_L
i64,str,str,i64,i64,i64,i64,i64
8473492,"""Matt""","""Beleskey""",1,0,0,0,1
8474009,"""Nick""","""Bonino""",1,0,0,0,1
8471699,"""Andrew""","""Cogliano""",1,0,0,0,1
8462041,"""Radek""","""Dvorak""",1,0,0,1,0
8475770,"""Emerson""","""Etem""",1,0,0,0,1


In [5]:
game_info_slim = (
    PBP_RAW
    .select('game_id', 'home_id', 'away_id', 'game_seconds', 'period_seconds', 'event_id', 'event_idx', 'event_type')
    .with_columns([
        (pl.col('game_id').cast(pl.Int64).alias('game_id')),
        (pl.col('home_id').cast(pl.Int64).alias('home_id')),
        (pl.col('away_id').cast(pl.Int64).alias('away_id')),
        (pl.col('game_seconds').cast(pl.Int64).alias('game_seconds')),
        (pl.col('event_id').cast(pl.Int64).alias('event_id')),
        (pl.col('event_idx').cast(pl.Int64).alias('event_idx'))
    ])
)

game_info_slim.sort('game_id','game_seconds', descending = False).head()

game_id,home_id,away_id,game_seconds,period_seconds,event_id,event_idx,event_type
i64,i64,i64,i64,i64,i64,i64,str
2012020001,4,5,0,0,51,5,"""PERIOD_START"""
2012020001,4,5,0,0,52,6,"""FACEOFF"""
2012020001,4,5,12,12,5,7,"""HIT"""
2012020001,4,5,29,29,53,8,"""SHOT"""
2012020001,4,5,29,29,6,9,"""STOPPAGE"""


In [12]:
# Shift Data
# Load Game ID and Home/Away Ids
g_shift_id = 2021020001
shift_link = "https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId="+str(g_shift_id)
shift_response = requests.get(shift_link)
shift_raw = pd.json_normalize(shift_response.json())
shift_raw = pd.json_normalize(shift_raw['data'])


# Create an empty DataFrame to store the normalized plays
normalized_shift = pd.DataFrame()

# Iterate over each row in plays_1 and normalize the JSON data
for _, row in shift_raw.iterrows():
    # Normalize the JSON data in the current row
    normalized_row_g = pd.json_normalize(row)

    # Concatenate the normalized row to the result DataFrame
    normalized_shift = pd.concat([normalized_shift, normalized_row_g], ignore_index=True)

# Create Columns From Data (Names and Shift Starts)
normalized_shift['player_name'] = normalized_shift['firstName'] + ' ' + normalized_shift['lastName']
# Period Time
normalized_shift['period_start_seconds'] = pd.to_datetime(normalized_shift['startTime'], format='%M:%S').dt.minute * 60 + pd.to_datetime(normalized_shift['startTime'], format='%M:%S').dt.second
normalized_shift['period_end_seconds'] = pd.to_datetime(normalized_shift['endTime'], format='%M:%S').dt.minute * 60 + pd.to_datetime(normalized_shift['endTime'], format='%M:%S').dt.second
# Game Time
normalized_shift['game_start_seconds'] = ( pd.to_datetime(normalized_shift['startTime'], format='%M:%S').dt.minute * 60 + pd.to_datetime(normalized_shift['startTime'], format='%M:%S').dt.second) + ((normalized_shift['period'] - 1) * 1200)
normalized_shift['game_end_seconds'] = ( pd.to_datetime(normalized_shift['endTime'], format='%M:%S').dt.minute * 60 + pd.to_datetime(normalized_shift['endTime'], format='%M:%S').dt.second) + ((normalized_shift['period'] - 1) * 1200)

# Rename
normalized_shift = normalized_shift.rename(columns = {
    'gameId': 'game_id',
    'id': 'shift_id',
    'playerId': 'player_id',
    'teamId': 'team_id',
    'shiftNumber': 'shift_number',
    'teamAbbrev': 'team_abbr'
})

# Keep
shift_keep_cols = ['game_id', 'shift_id', 'team_id', 'player_id', 'player_name', 'period',
                   'period_start_seconds', 'period_end_seconds', 'game_start_seconds', 'game_end_seconds',
                   'eventNumber', 'team_abbr', 'shift_number', 'typeCode']
normalized_shift = normalized_shift[shift_keep_cols]
normalized_shift = pl.DataFrame(normalized_shift)

# Join To Get Home Team
normalized_shift = (
    normalized_shift
    .join(game_info_slim.drop('game_seconds', 'period_seconds', 'event_id', 'event_idx', 'event_type').unique(), on='game_id', how='left')
    .with_columns(pl.when(pl.col('home_id') == pl.col('team_id')).then(pl.lit('home')).otherwise(pl.lit('away')).alias('team_type'))
    .drop('home_id', 'away_id')
    .unique()
)

# Join To Separate Goalies
normalized_shift = (
    normalized_shift
    .join(ROSTER_DF.with_columns([
        (pl.col('player_id').cast(pl.Int64).alias('player_id')),
        (pl.col('pos_G').cast(pl.Int64).alias('pos_G'))
    ])
    .select('player_id', 'pos_G'), on='player_id', how='left')
    .unique()
)


# Group by 'player_id'
grouped_shifts = normalized_shift.select('player_id', 'game_id', 'period', 'team_type', 'pos_G', 'period_start_seconds', 'period_end_seconds').to_pandas()
grouped_shifts = grouped_shifts.groupby(['game_id', 'period', 'period_start_seconds', 'period_end_seconds', 'team_type', 'pos_G'])

# Aggregate using the agg method
result_df = grouped_shifts.agg(
    player_id_list=('player_id', list),
).reset_index()

seconds_df = PBP_RAW.select(pl.col('game_id').cast(pl.Int64),pl.col('period').cast(pl.Int64), pl.col('period_seconds').cast(pl.Int64), 'event_id').filter(pl.col('game_id') == 2021020001).to_pandas()


# Function to get player lists based on conditions
def get_player_lists(row, type, pos):
    p_secs = row['period_seconds']
    per = row['period']
    
    # Adjust conditions as needed
    condition = (
        (result_df['period'] == per) &
        (result_df['period_start_seconds'] <= p_secs) &
        (result_df['period_end_seconds'] > p_secs) &
        (result_df['team_type'] == type) &
        (result_df['pos_G'] == pos)
    )
    
    selected_rows = result_df[condition]
    
    if not selected_rows.empty:
        # Combine player lists
        combined_players = [player_id for player_list in selected_rows['player_id_list'] for player_id in player_list]
        return combined_players
    else:
        return []

# Apply the function to each row of seconds_df
seconds_df['home'] = seconds_df.apply(get_player_lists,type='home', pos=0, axis=1)
seconds_df['away'] = seconds_df.apply(get_player_lists,type='away', pos=0, axis=1)
seconds_df['home_goalie'] = seconds_df.apply(get_player_lists, type='home', pos=1, axis=1)
seconds_df['away_goalie'] = seconds_df.apply(get_player_lists, type='away', pos=1, axis=1)

# Columns with lists of player IDs
list_columns = ["home", "away", "home_goalie", "away_goalie"]

# Iterate over each list column
for column in list_columns:
    # Iterate over each row
    for index, row in seconds_df.iterrows():
        # Extract the list of player IDs
        player_ids = row[column]
        
        # Iterate over the player IDs in the list
        for i, player_id in enumerate(player_ids):
            # Create a new column name
            new_column_name = f"{column}_{i+1}_on"
            
            # Create a new column in the DataFrame
            seconds_df.at[index, new_column_name] = player_id

# Drop unnecessary Columns
seconds_df = seconds_df.drop(['home', 'away', 'home_goalie', 'away_goalie'], axis=1)
# Display the updated DataFrame
print(seconds_df.head())


      game_id  period  period_seconds  event_id  home_1_on  home_2_on  \
0  2021020001       1               0      51.0  8476292.0  8478010.0   
1  2021020001       1               0      52.0  8476292.0  8478010.0   
2  2021020001       1              18       8.0  8476292.0  8478010.0   
3  2021020001       1              38       9.0  8475167.0  8480172.0   
4  2021020001       1              38      53.0  8475167.0  8480172.0   

   home_3_on  home_4_on  home_5_on  home_6_on  away_1_on  away_2_on  \
0  8478416.0  8476453.0  8474151.0        NaN  8475208.0  8471724.0   
1  8478416.0  8476453.0  8474151.0        NaN  8475208.0  8471724.0   
2  8478416.0  8476453.0  8474151.0        NaN  8475208.0  8471724.0   
3  8473986.0  8478519.0  8474564.0        NaN  8477969.0  8478507.0   
4  8473986.0  8478519.0  8474564.0        NaN  8477969.0  8478507.0   

   away_3_on  away_4_on  away_5_on  home_goalie_1_on  away_goalie_1_on  
0  8475810.0  8470604.0  8478046.0         8476883.0         

In [8]:
# Group by 'player_id'
grouped_shifts = normalized_shift.select('player_id', 'game_id', 'period', 'team_type', 'pos_G', 'period_start_seconds', 'period_end_seconds').to_pandas()
grouped_shifts = grouped_shifts.groupby(['game_id', 'period', 'period_start_seconds', 'period_end_seconds', 'team_type', 'pos_G'])

# Aggregate using the agg method
result_df = grouped_shifts.agg(
    player_id_list=('player_id', list),
).reset_index()
# Show the result DataFrame
print(result_df.head())

      game_id  period  period_start_seconds  period_end_seconds team_type  \
0  2021020001       1                     0                  26      away   
1  2021020001       1                     0                  29      home   
2  2021020001       1                     0                  32      away   
3  2021020001       1                     0                1200      away   
4  2021020001       1                     0                1200      home   

   pos_G                                 player_id_list  
0      0                             [8475208, 8471724]  
1      0  [8476453, 8478416, 8476292, 8474151, 8478010]  
2      0                    [8475810, 8470604, 8478046]  
3      1                                      [8477465]  
4      1                                      [8476883]  


In [10]:
seconds_df = PBP_RAW.select(pl.col('game_id').cast(pl.Int64),pl.col('period').cast(pl.Int64), pl.col('period_seconds').cast(pl.Int64), 'event_id').filter(pl.col('game_id') == 2021020001).to_pandas()


#p_secs = 22
#per = 1
#print(result_df[(result_df['period'] == per) & (result_df['period_start_seconds'] <= p_secs) & (result_df['period_end_seconds'] > p_secs)])

# Function to get player lists based on conditions
def get_player_lists(row, type, pos):
    p_secs = row['period_seconds']
    per = row['period']
    
    # Adjust conditions as needed
    condition = (
        (result_df['period'] == per) &
        (result_df['period_start_seconds'] <= p_secs) &
        (result_df['period_end_seconds'] > p_secs) &
        (result_df['team_type'] == type) &
        (result_df['pos_G'] == pos)
    )
    
    selected_rows = result_df[condition]
    
    if not selected_rows.empty:
        # Combine player lists
        combined_players = [player_id for player_list in selected_rows['player_id_list'] for player_id in player_list]
        return combined_players
    else:
        return []

# Apply the function to each row of seconds_df
seconds_df['home'] = seconds_df.apply(get_player_lists,type='home', pos=0, axis=1)
seconds_df['away'] = seconds_df.apply(get_player_lists,type='away', pos=0, axis=1)
seconds_df['home_goalie'] = seconds_df.apply(get_player_lists, type='home', pos=1, axis=1)
seconds_df['away_goalie'] = seconds_df.apply(get_player_lists, type='away', pos=1, axis=1)

# Columns with lists of player IDs
list_columns = ["home", "away", "home_goalie", "away_goalie"]

# Iterate over each list column
for column in list_columns:
    # Iterate over each row
    for index, row in seconds_df.iterrows():
        # Extract the list of player IDs
        player_ids = row[column]
        
        # Iterate over the player IDs in the list
        for i, player_id in enumerate(player_ids):
            # Create a new column name
            new_column_name = f"{column}_{i+1}_on"
            
            # Create a new column in the DataFrame
            seconds_df.at[index, new_column_name] = player_id

# Drop unnecessary Columns
seconds_df = seconds_df.drop(['home', 'away', 'home_goalie', 'away_goalie'], axis=1)
# Display the updated DataFrame
print(seconds_df.head())



      game_id  period  period_seconds  event_id  home_1_on  home_2_on  \
0  2021020001       1               0      51.0  8476453.0  8478416.0   
1  2021020001       1               0      52.0  8476453.0  8478416.0   
2  2021020001       1              18       8.0  8476453.0  8478416.0   
3  2021020001       1              38       9.0  8475167.0  8480172.0   
4  2021020001       1              38      53.0  8475167.0  8480172.0   

   home_3_on  home_4_on  home_5_on  home_6_on  away_1_on  away_2_on  \
0  8476292.0  8474151.0  8478010.0        NaN  8475208.0  8471724.0   
1  8476292.0  8474151.0  8478010.0        NaN  8475208.0  8471724.0   
2  8476292.0  8474151.0  8478010.0        NaN  8475208.0  8471724.0   
3  8473986.0  8474564.0  8478519.0        NaN  8478507.0  8477969.0   
4  8473986.0  8474564.0  8478519.0        NaN  8478507.0  8477969.0   

   away_3_on  away_4_on  away_5_on  home_goalie_1_on  away_goalie_1_on  
0  8475810.0  8470604.0  8478046.0         8476883.0         

In [148]:
seconds_df[~seconds_df['away_11_on'].isna()].head()

,game_id,period,period_seconds,event_id,home,away,home_goalie,away_goalie,home_1_on,home_2_on,home_3_on,home_4_on,home_5_on,home_6_on,home_7_on,home_8_on,home_9_on,home_10_on,home_11_on,home_12_on,away_1_on,away_2_on,away_3_on,away_4_on,away_5_on,away_6_on,away_7_on,away_8_on,away_9_on,away_10_on,away_11_on,home_goalie_1_on,away_goalie_1_on
137,2021020001,2,251,270.0,"[8479410, 8474567, 8474564, 8478519, 8479390, ...","[8478043, 8478507, 8470619, 8482055, 8477969, ...",[8476883],[8477465],8479410.0,8474567.0,8474564.0,8478519.0,8479390.0,8470621.0,8477930.0,8475167.0,8480172.0,NaN,NaN,NaN,8478043.0,8478507.0,8470619.0,8482055.0,8477969.0,8470619.0,8471724.0,8478866.0,8475208.0,8476934.0,8476927.0,8476883.0,8477465.0
138,2021020001,2,251,271.0,"[8479410, 8474567, 8474564, 8478519, 8479390, ...","[8478043, 8478507, 8470619, 8482055, 8477969, ...",[8476883],[8477465],8479410.0,8474567.0,8474564.0,8478519.0,8479390.0,8470621.0,8477930.0,8475167.0,8480172.0,NaN,NaN,NaN,8478043.0,8478507.0,8470619.0,8482055.0,8477969.0,8470619.0,8471724.0,8478866.0,8475208.0,8476934.0,8476927.0,8476883.0,8477465.0
292,2021020001,3,893,662.0,"[8476453, 8478010, 8478519, 8475167, 8479410, ...","[8471724, 8475208, 8476934, 8476927, 8478866, ...",[8476883],[8477465],8476453.0,8478010.0,8478519.0,8475167.0,8479410.0,8474564.0,8474567.0,8474151.0,8477930.0,8479390.0,8470621.0,NaN,8471724.0,8475208.0,8476934.0,8476927.0,8478866.0,8476927.0,8470604.0,8475810.0,8478046.0,8478507.0,8477969.0,8476883.0,8477465.0
293,2021020001,3,893,663.0,"[8476453, 8478010, 8478519, 8475167, 8479410, ...","[8471724, 8475208, 8476934, 8476927, 8478866, ...",[8476883],[8477465],8476453.0,8478010.0,8478519.0,8475167.0,8479410.0,8474564.0,8474567.0,8474151.0,8477930.0,8479390.0,8470621.0,NaN,8471724.0,8475208.0,8476934.0,8476927.0,8478866.0,8476927.0,8470604.0,8475810.0,8478046.0,8478507.0,8477969.0,8476883.0,8477465.0
304,2021020001,3,1055,672.0,"[8479525, 8474034, 8479410, 8474151, 8470621, ...","[8478542, 8476927, 8477969, 8476934, 8478507, ...",[8476883],[8477465],8479525.0,8474034.0,8479410.0,8474151.0,8470621.0,8476292.0,8473986.0,8476453.0,8478010.0,NaN,NaN,NaN,8478542.0,8476927.0,8477969.0,8476934.0,8478507.0,8478542.0,8478017.0,8477244.0,8475810.0,8470604.0,8478046.0,8476883.0,8477465.0


In [140]:
seconds_df = PBP_RAW.select(pl.col('game_id').cast(pl.Int64),pl.col('period').cast(pl.Int64), pl.col('period_seconds').cast(pl.Int64)).filter(pl.col('game_id') == 2021020001).to_pandas()


# Merge data frames based on conditions
merged_df = result_df.merge(seconds_df, on=["game_id", "period"], how="inner")

# Apply additional conditions
merged_df = merged_df[
    (merged_df["period_start_seconds"] <= merged_df["period_seconds"]) &
    (merged_df["period_end_seconds"] >= merged_df["period_seconds"])
]

#print(merged_df.sort_values(by='period_seconds', ascending=True).head(20))

# Assuming merged_df is your pandas DataFrame
aggregated_df = merged_df.groupby(["game_id", "period", 'period_seconds', "team_type", "pos_G"]).agg(
    player_id_list=("player_id_list", list)
).reset_index()

print(aggregated_df.head())

# Flatten the list column
#aggregated_df["home_skaters_on"] = aggregated_df[(aggregated_df['team_type'] == 'home') & (aggregated_df['pos_G'] == 0)]["player_id_list"].apply(lambda x: [item for sublist in x for item in sublist])
#aggregated_df["away_skaters_on"] = aggregated_df[(aggregated_df['team_type'] == 'away') & (aggregated_df['pos_G'] == 0)]["player_id_list"].apply(lambda x: [item for sublist in x for item in sublist])
#aggregated_df["home_goalie_on"] = aggregated_df[(aggregated_df['team_type'] == 'home') & (aggregated_df['pos_G'] == 1)]["player_id_list"].apply(lambda x: [item for sublist in x for item in sublist])
#aggregated_df["away_goalie_on"] = aggregated_df[(aggregated_df['team_type'] == 'away') & (aggregated_df['pos_G'] == 1)]["player_id_list"].apply(lambda x: [item for sublist in x for item in sublist])

# Drop the original list column if needed
#aggregated_df = aggregated_df.drop(columns=["player_id_list"])

#print(len(aggregated_df['combined_players'][0]))

# Display the result
#print(aggregated_df.head())

      game_id  period  period_seconds team_type  pos_G  \
0  2021020001       1               0      away      0   
1  2021020001       1               0      away      1   
2  2021020001       1               0      home      0   
3  2021020001       1               0      home      1   
4  2021020001       1              18      away      0   

                                      player_id_list  
0  [[8471724, 8475208], [8471724, 8475208], [8475...  
1                             [[8477465], [8477465]]  
2  [[8476292, 8478416, 8476453, 8474151, 8478010]...  
3                             [[8476883], [8476883]]  
4  [[8471724, 8475208], [8475810, 8478046, 8470604]]  


In [76]:
seconds_df = PBP_RAW.select(pl.col('game_id').cast(pl.Int64),pl.col('game_seconds').cast(pl.Int64), 'event_id')


cond = normalized_shift.filter((pl.col('team_type') == 'home') & (pl.col('pos_G') == 1)).select(
    pl.format(
        "WHEN game_seconds >= {} AND game_seconds < {} THEN '{}'",
        pl.col("game_start_seconds"),
        pl.col("game_end_seconds"),
        pl.col("player_id"),
    ).alias("cond")
)
cond = "\n".join(cond["cond"]) + " ELSE 'Not Found'"

print(cond)


ctxt = pl.SQLContext()
ctxt.register("seconds_df", seconds_df.lazy())  # <-- `df_pl` is your first dataframe

print(
    ctxt.execute(
        f"""\
    SELECT game_id, game_seconds, event_id,
    CASE
        {cond}
    END AS home_goalie
    FROM seconds_df
""",
        eager=True,
    )
)

WHEN game_seconds >= 1200 AND game_seconds < 2400 THEN '8476883'
WHEN game_seconds >= 2400 AND game_seconds < 3228 THEN '8476883'
WHEN game_seconds >= 3378 AND game_seconds < 3387 THEN '8476883'
WHEN game_seconds >= 3263 AND game_seconds < 3283 THEN '8476883'
WHEN game_seconds >= 3293 AND game_seconds < 3372 THEN '8476883'
WHEN game_seconds >= 3455 AND game_seconds < 3469 THEN '8476883'
WHEN game_seconds >= 3491 AND game_seconds < 3600 THEN '8476883'
WHEN game_seconds >= 0 AND game_seconds < 1200 THEN '8476883'
WHEN game_seconds >= 3409 AND game_seconds < 3424 THEN '8476883' ELSE 'Not Found'


KeyboardInterrupt: 